In [1]:
from selenium import webdriver
import time
import re
from urllib.request import urlopen
import json
from pandas.io.json import json_normalize
import pandas as pd, numpy as np
from bs4 import BeautifulSoup as bs
import os
import wget

In [2]:
### specify the values for the following variables and then run all the cells
ig_username = 'infostealer'
ig_password = 'infostealer1'
keyword = 'theomnidesk'

In [3]:
from selenium import webdriver
driver = webdriver.Chrome() #if your chromedriver.exe inside root
driver.get("http://www.instagram.com")

In [4]:
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
username = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='username']")))
password = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='password']")))

In [5]:
# Enter username, password, and press login button
username.clear()
username.send_keys(ig_username)
password.clear()
password.send_keys(ig_password)
Login_button = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))).click()

In [6]:
# click 'not now'
not_now = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/section/main/div/div/div/div/button'))).click()

In [7]:
# in the search box, search keyword then go to the first result

import time
searchbox = WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.XPATH, "//input[@placeholder='Search']")))
searchbox.clear()
searchbox.send_keys(keyword)
time.sleep(5)
searchbox.send_keys(Keys.ENTER)
time.sleep(5)
searchbox.send_keys(Keys.ENTER)
time.sleep(5)

In [8]:
# create a folder for pictures

path = os.getcwd()
path = os.path.join(path, keyword)
os.mkdir(path)

In [9]:
counter = 0
def scroll():

    global keyword

# Get scroll height
last_height = driver.execute_script("return document.body.scrollHeight")
while True:
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
    # Scrape images as we scroll
    driver.implicitly_wait(5)
    images = driver.find_elements_by_tag_name('img')
    images = [image.get_attribute('src') for image in images]
    images = images[:-3] #slicing-off IG logo and profile pictures
    print('Number of scraped images: ', len(images))
    
    # Download image as we scroll  
    for image in images:
        save_as = os.path.join(path, keyword[1:] + str(counter) +
        '.jpg')
        wget.download(image, save_as)
        counter += 1
    
    # Wait to load page     
    time.sleep(5)       
    # Scroll down so we do not scrape the same images
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")


     
    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    height_diff = new_height - last_height
    if new_height == last_height:
        break
    last_height = new_height

scroll()

Number of scraped images:  38
100% [..............................................................................] 64420 / 64420Number of scraped images:  44
100% [..............................................................................] 23820 / 23820Number of scraped images:  44
100% [..............................................................................] 53537 / 53537Number of scraped images:  44
100% [..............................................................................] 53007 / 53007Number of scraped images:  44
100% [..............................................................................] 56823 / 56823Number of scraped images:  44
100% [..............................................................................] 66046 / 66046Number of scraped images:  44
100% [..............................................................................] 54998 / 54998Number of scraped images:  44
100% [.............................................................

StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=92.0.4515.159)
  (Driver info: chromedriver=2.38.552522 (437e6fbedfa8762dec75e2c5b3ddb86763dc9dcb),platform=Windows NT 10.0.19043 x86_64)


In [ ]:
# take a break
time.sleep(5) 

# Scroll to top
driver.execute_script("window.scrollTo(0, 0);")

# take another break
time.sleep(5) 

#Extract links from user profile page
links=[]
source = driver.page_source
data=bs(source, 'html.parser')
links = data.find_all('div')

In [ ]:
import re
listToStr = ' '.join([str(link) for link in links])
links = re.findall(r'<div class="v1Nh3 kIKUG _bz0w"><a href="/p/(.{11})/" tabindex="0">',listToStr)
links

In [ ]:
string = 'https://www.instagram.com/p/'
links2 = [string + link for link in links]

# loop through all the post links collected
for j in range(len(links2)):

    # For one iteration
    # store each post's html and retrieve image, num_likes, caption and comments
    driver.get(links2[j])
    source = driver.page_source
    data=bs(source, 'html.parser')
    print("\n----------Post----------\n")
    print(links2[j])

    # pull out the captions
    captionsToStr = data.find_all('div')
    captionsToStr = ' '.join([str(c) for c in captionsToStr])
    captions = re.findall(r'<div class="C4VMK">(.+)</div>',captionsToStr)
    cStr = re.findall(r'<span class="">(.+)</span>',str(captions))
    text = str(cStr[0]).split("</span>")
    copy = text[0]
    print(copy)

    # pull out the number of likes
    likesToStr = data.find_all('a')
    likes = re.findall(r'<span>(.+)</span>',str(likesToStr))
    print(likes)

    # pull out the comments | comments[0] is the caption
    comments = data.find_all('div', class_='C4VMK')
    for i in range(len(comments)):
        # retrieve the first comment
        if i>0 :
            comment = bs(str(comments[i]), 'html.parser').find_all('span', class_='')
            print(comment)

In [ ]:
len(links)

In [ ]:
# copy.split('tabindex="0"')
# copy.split()